In [80]:
import pandas as pd

df = pd.read_csv('../data/curated/Q3engineered-data.csv')


Affordability

In [81]:
dfaff = df.groupby('suburb').agg({'price':'mean','bedrooms':'mean'}).reset_index()
dfaff['Affordability'] = dfaff['price'] / dfaff['bedrooms']
dfaff = dfaff.sort_values(by='Affordability')

In [82]:
dfaff['suburb'][0:5]

178         Mambourin
179       Manor Lakes
224          Plumpton
275    Thornhill Park
302      Wyndham Vale
Name: suburb, dtype: object

Liveability

In [83]:
df['overall_icsea']=df['min_pri_icsea']+df['min_sec_icsea']

In [84]:
icsea_list=[df['overall_icsea'].quantile(0.33),df['overall_icsea'].quantile(0.66)]

In [85]:
crime_list=[df['Crime Rate'].quantile(0.33),df['Crime Rate'].quantile(0.66)]

In [86]:
def calculate_livability(row):
    sum = 0
    # park walkability
    if row['park_walkability'] == '$<$1.5 km':
        sum+=1
    elif row['park_walkability'] == '1.5-5km':
        sum += 0.5
    # School walkbility
    if row['is_primary_school_walkable'] == 'Yes': sum += 0.5
    if row['is_secondary_school_walkable'] == 'Yes': sum += 0.5
    # Public transport walkbility
    if row['is_train_station_walkable'] == '$<$1.5 km':
        sum+=0.7
    elif row['is_train_station_walkable'] == '1.5-5km':
        sum += 0.35
    if row['is_tram_walkable'] == '$<$1.5 km':
        sum+=0.2
    elif row['is_tram_walkable'] == '1.5-5km':
        sum += 0.1
    if row['is_bus_walkable'] == '$<$1.5 km':
        sum+=0.1
    elif row['is_bus_walkable'] == '1.5-5km':
        sum += 0.05
    # School rating
    if row['overall_icsea'] > icsea_list[1]:
        sum+=1
    elif row['overall_icsea'] > icsea_list[0]:
        sum+=0.5
    # Crime number
    if row['Crime Rate'] < crime_list[0]:
        sum+=1
    elif row['Crime Rate'] < crime_list[1]:
        sum+=0.5

    return sum


In [87]:
df['livability']=df.apply(calculate_livability, axis=1)

In [88]:
dfliv = df.groupby('suburb').agg({'livability':'mean'}).reset_index()
dfliv = dfliv.sort_values(by='livability',ascending=False)

In [90]:
dfliv['suburb'][0:5]

229     Princes Hill
4        Albert Park
119    Fitzroy North
137         Hawthorn
57        Canterbury
Name: suburb, dtype: object